In [2]:
import json, csv, os
from csv import writer, reader

In [2]:
def save2csv(filename, header, data):
    with open(filename, 'w', newline='') as csvfile:
        csvw = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        csvw.writerow(header)
        for d in data:
            csvw.writerow(d)

In [3]:
def save2json(filename, dump):
    out_file = open(filename, "w")
    json.dump(dump, out_file)
    out_file.close()

In [ ]:
ts = set()

In [ ]:
with open('imgflip.json') as jsonfile:
    ist =  json.load(jsonfile)
    for ii in ist:
        ts.add(ii['template_ID'])

In [ ]:
ls = list(map(lambda x: [x,'https://imgflip.com/meme/' + x],list(ts)))

In [ ]:
save2csv('imgflip_templates2.csv',['template','templ'],ls)

In [ ]:
dump = {}
with open('aliases.csv') as aliasfile:
    aliases = reader(aliasfile)
    for a in aliases:
        dump[a[1]]=a[0]
save2json('aliases.json',dump)

In [ ]:
cleaned = {}
with open('aliases.json') as aliasfile:
    alias_map = json.load(aliasfile)
    with open('imgflip.json') as jsonfile:
        ist =  json.load(jsonfile)
        for ii in ist:
            if(ii['URL']==None or ii['alt_text']==None):
                continue
            ii['turl'] = alias_map[ii['template_ID']]
            cleaned[ii['URL']] = ii
            
save2json('imgflip.cleaned.2.json',cleaned)

In [ ]:
ist[0]

In [ ]:
{'template_title': 'Is This A Pigeon',
 'template_ID': '100777631',
 'URL': '/i/6lv2qe',
 'title': 'What parents here',
 'author': '-Luigifan-',
 'view_count': '2,421',
 'upvote_count': '39',
 'comment_count': '1',
 'image_url': '//i.imgflip.com/6lv2qe.jpg',
 'alt_text': 'What parents here |  PARENTS; ANYTHING I SAY; HMMM. . . TASTES LIKE DISRESPECT | image tagged in memes,is this a pigeon | made w/ Imgflip meme maker',
 'turl': 'https://imgflip.com/meme/Is-This-A-Pigeon'}

In [ ]:
with open('imgflip.spotlight.json', 'r') as f2:
    spotlight = json.load(f2)
    keys = list(map(lambda x: [x], list(spotlight.keys())))
    save2csv('imgflip.spotlighted.csv', ['URL'], keys)

In [ ]:
num = 0
memes = []
with open('imgflip.cleaned.json', 'r', encoding='utf8') as f1:
    for m in json.load(f1):
        if(m['URL'] in keys or (m['URL']==None) or m['alt_text']==None):
            continue
        else:
            memes.append({'URL': m['URL'], 'alt_text':m['alt_text']})
            # if(len(memes) == 10000):
            #     save2json('shrinked.'+str(num)+'.json',memes)
            #     num = num + 1
            #     memes = []
save2json('imgflip.cleaned2.json',ist)

In [ ]:
len(memes)

In [ ]:
save2json('shrinked.json',memes[1:10000])

In [ ]:
memes

In [4]:
f1 = open('imgflip.cleaned.json', 'r', encoding='utf8')
j1 = json.load(f1)

In [7]:
j1["/i/6lv2qe"]

{'template_title': 'Is This A Pigeon',
 'template_ID': '100777631',
 'URL': '/i/6lv2qe',
 'title': 'What parents here',
 'author': '-Luigifan-',
 'view_count': '2,421',
 'upvote_count': '39',
 'comment_count': '1',
 'image_url': '//i.imgflip.com/6lv2qe.jpg',
 'alt_text': 'What parents here |  PARENTS; ANYTHING I SAY; HMMM. . . TASTES LIKE DISRESPECT | image tagged in memes,is this a pigeon | made w/ Imgflip meme maker',
 'turl': 'https://imgflip.com/meme/Is-This-A-Pigeon'}

In [14]:
by_template = {}
for k in j1.keys():
    meme = j1[k]
    print(by_template)
    if(meme['template_ID'] in by_template.keys()):
        
        by_template[meme['template_ID']].append(meme)
    else:
        by_template[meme['template_ID']] = [meme]

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [4]:
import json, os

In [3]:
memes_entities = []
with open('../imkg/dbpedia-wikidata.json') as jsonfile:
    db2wdmapping = json.load(jsonfile)
    for file in os.listdir('./data prep/imgflip.spotlight.bytemplate'):
        outFname = './data prep/imgflip.spotlight.bytemplate/kym/'+file.replace(file,"kym."+file)
        if (os.path.isfile(os.path.join('./data prep/imgflip.spotlight.bytemplate', file)) and file.endswith("json") and not os.path.exists(os.path.join('./data prep/imgflip.spotlight.bytemplate/kym', file))):
            with open(outFname, 'w') as out_file:
                fname = './data prep/imgflip.spotlight.bytemplate/'+str(file)
                spotlight_entities = open(fname)
                jse = json.load(spotlight_entities)
                for je in jse:
                    if("Resources" in je.keys()):
                        for r in je["Resources"]:
                            res = r['@URI'].replace('http://dbpedia.org/resource/','')
                            if(res in db2wdmapping.keys()):
                                r['QID'] = 'https://www.wikidata.org/wiki/'+db2wdmapping[res]
                                if(type(r["@types"])==str):
                                    ls = r["@types"].replace("DUL:", "http://www.loa-cnr.it/ontologies/DOLCE-Lite#").replace("Wikidata:", "https://www.wikidata.org/wiki/").replace("Schema:", "https://schema.org/").split(",")
                                    ls = list(filter(None, ls))
                                    ls = [l for l in ls if 'DBpedia' not in l]
                                elif(type(r["@types"])==list):
                                    ls = list(map(lambda s: s.replace("DUL:", "http://www.loa-cnr.it/ontologies/DOLCE-Lite#").replace("Wikidata:", "https://www.wikidata.org/wiki/").replace("Schema:", "https://schema.org/").replace("DBpedia:", ""),r["@types"]))
                                    errors.append([row[1],res,r["@types"]])
                               #Removing dbpedia types that do not have a correspondence in wikidata
                                for l in ls:
                                    if(l in db2wdmapping.keys()):
                                        l = 'https://www.wikidata.org/wiki/'+db2wdmapping[l]     
                                r["@typeList"]=ls
                memes_entities.append(je)
                spotlight_entities.close()
                json.dump(memes_entities, out_file, indent = 6)


KeyboardInterrupt



In [21]:
for file in os.listdir('./data prep/imgflip.cleaned.bytemplate'):
    if (file.endswith(".json")):
        with open('./data prep/imgflip.cleaned.bytemplate/'+file, "r") as jfile:
            jj = json.load(jfile) 
            for j in jj:
                # j['URL']= "http://imgflip.com"+j['URL']
                j['URL']= j['URL'].replace('http','https')
                # j['image_url'] = 'https:'+ j['image_url']
            out_file = open('./data prep/imgflip.cleaned.bytemplate/'+ file, 'w')
            json.dump(jj,out_file)
            out_file.close()

In [24]:
for file in os.listdir('./data prep/imgflip.spotlight.bytemplate/'):
    if (file.endswith(".json")):
        with open('./data prep/imgflip.spotlight.bytemplate/'+file, "r") as jfile:
            jj = json.load(jfile) 
            for j in jj:
                j['instance_ID']= "https://imgflip.com"+j['instance_ID']
                # j['image_url'] = 'https:'+ j['image_url']
            out_file = open('./data prep/imgflip.spotlight.bytemplate/'+ file, 'w')
            json.dump(jj,out_file)
            out_file.close()

NameError: name 'affo' is not defined